In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir

# fine-tune
# 1. save models (third party/myself)
# 2. restore models checkpoints (断点恢复)
# 3. keep some layers fixed.


In [3]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[ 0.00392157  0.02745098  0.09019608 ... -0.28627451 -0.29411765
  -0.30980392]
 [-0.48235294 -0.41176471 -0.30980392 ... -0.21568627 -0.29411765
  -0.44313725]
 [ 0.98431373  0.99215686  0.97647059 ... -0.65490196 -0.64705882
  -0.63921569]
 ...
 [ 0.88235294  0.85098039  0.85098039 ...  0.51372549  0.54509804
   0.58431373]
 [-0.20784314 -0.19215686 -0.17647059 ...  0.67843137  0.63921569
   0.59215686]
 [ 0.90588235  0.92941176  0.97647059 ...  0.12156863  0.0745098
   0.05882353]] [9 9 9 0 7 3 9 3 9 1]


In [4]:
x = tf.placeholder(tf.float32, [None, 3072])

# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# vgg-net思想
# acc: 0.73150
conv1_1 = tf.layers.conv2d(x_image,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        trainable = False,
                        activation=tf.nn.relu,
                        name='conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        trainable = False,
                        activation=tf.nn.relu,
                        name='conv1_2')

# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool1')

conv2_1 = tf.layers.conv2d(pooling1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        trainable = False,
                        activation=tf.nn.relu,
                        name='conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        trainable = False,
                        activation=tf.nn.relu,
                        name='conv2_2')

# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv3_1')

conv3_2 = tf.layers.conv2d(conv3_1,
                        32, # output channel number
                        (3,3), # kernel size
                        padding = 'same', 
                        activation=tf.nn.relu,
                        name='conv3_2')
# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                  (2,2), # kernel size
                                  (2,2), # stride
                                  name='pool3')

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0717 20:59:58.594243 12628 deprecation.py:323] From <ipython-input-4-567957893c51>:18: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0717 20:59:58.602027 12628 deprecation.py:506] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0717 20:59:59.214317 12628 deprecation.py:323] From <ipython-input-4-567957893c51>:32: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0717 20:59:59.584749 12628 deprecation.py:323] From <ipython-input-4-56795789

In [5]:
def variable_summary(var, name):
    """Constructs summary fro statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
        
# 1. 指定面板图上显示的变量
loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image('inputs_image', source_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

# 2. 训练过程中将这些变量计算出来，输出到文件中
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')

if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [6]:
# 1. save models (third_party)
model_dir = os.path.join(run_dir, 'model')
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

saver = tf.train.Saver()

In [7]:
# 2. restroe models checkpoint
model_name = 'ckp-01000'
model_path = os.path.join(model_dir, model_name)

In [8]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps =100

output_summary_every_steps = 100
output_model_every_steps = 100

with tf.Session() as sess:
    sess.run(init)
    # 2. 训练过程中将这些变量计算出来，输出到文件中
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = train_data.next_batch(batch_size)
    
    if os.path.exists(model_path + '.index'):
        saver.restore(sess, model_path)
        print('model restored from %s' % model_path)
    else:
        print('model %s does not exist' % model_path)
    
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        
        eval_ops_results = sess.run(
            eval_ops, 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        loss_val, accu_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test],
                                           feed_dict={
                                               x: fixed_test_batch_data,
                                               y: fixed_test_batch_labels
                                           })[0]
            test_writer.add_summary(test_summary_str, i+1)
        if (i + 1) % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if (i + 1) % 500 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })[0]
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 
        if (i + 1) % output_model_every_steps == 0:
            saver.save(sess, os.path.join(model_dir, 'ckp-%05d' % (i + 1)))
            print('model saved to ckp-%05d' % (i + 1))

model .\run_vgg_tensorboard\model\ckp-01000 does not exist
[Train] step: 99, loss: 1.86260, acc:0.35000
model saved to ckp-00100
[Train] step: 199, loss: 1.95509, acc:0.30000
model saved to ckp-00200
[Train] step: 299, loss: 1.91498, acc:0.35000
model saved to ckp-00300
[Train] step: 399, loss: 1.88190, acc:0.25000
model saved to ckp-00400
[Train] step: 499, loss: 1.82918, acc:0.35000
(10000, 3072) (10000,)
[Test] Step: 500, acc: 0.37950
model saved to ckp-00500


W0717 21:00:17.707630 12628 deprecation.py:323] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\training\saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


[Train] step: 599, loss: 1.91650, acc:0.25000
model saved to ckp-00600
[Train] step: 699, loss: 1.22223, acc:0.55000
model saved to ckp-00700
[Train] step: 799, loss: 2.07750, acc:0.40000
model saved to ckp-00800
[Train] step: 899, loss: 1.43353, acc:0.55000
model saved to ckp-00900
[Train] step: 999, loss: 1.56719, acc:0.25000
(10000, 3072) (10000,)
[Test] Step: 1000, acc: 0.42250
model saved to ckp-01000
[Train] step: 1099, loss: 1.51716, acc:0.40000
model saved to ckp-01100
[Train] step: 1199, loss: 1.49178, acc:0.55000
model saved to ckp-01200
[Train] step: 1299, loss: 1.49813, acc:0.30000
model saved to ckp-01300
[Train] step: 1399, loss: 1.56301, acc:0.40000
model saved to ckp-01400
[Train] step: 1499, loss: 1.91774, acc:0.35000
(10000, 3072) (10000,)
[Test] Step: 1500, acc: 0.43900
model saved to ckp-01500
[Train] step: 1599, loss: 1.10856, acc:0.70000
model saved to ckp-01600
[Train] step: 1699, loss: 1.11530, acc:0.55000
model saved to ckp-01700
[Train] step: 1799, loss: 1.361

In [9]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Test] Step: 10000, acc: 0.57400
